In [1]:
import pickle
import pandas as pd
import numpy as np

## Re-open Old Pickles

In [2]:
with open('offense.pickle', 'rb') as f:
    offense = pickle.load(f)

In [3]:
with open('defense.pickle', 'rb') as f:
    defense = pickle.load(f)

In [4]:
offense.head()

,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,playerID,position
4,1,0,0,49,0,0,Emmanuel Sanders,WR
5,2,0,0,39,0,0,Emmanuel Sanders,WR
6,3,0,0,117,0,2,Emmanuel Sanders,WR
7,4,0,0,88,0,1,Emmanuel Sanders,WR
8,5,0,0,80,0,0,Emmanuel Sanders,WR


In [5]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
0,Arizona Cardinals,1,2.0,0,0,0,0
1,Arizona Cardinals,2,2.0,2,0,0,0
2,Arizona Cardinals,3,2.0,1,0,1,0
3,Arizona Cardinals,4,2.0,0,0,0,0
4,Arizona Cardinals,5,2.0,0,0,0,0


## Create Users

In [6]:
names = pd.read_csv('names.csv')

In [7]:
names = names[['FirstName', 'Surname']]

In [8]:
names.head()

,FirstName,Surname
0,James,Smith
1,Michael,Smith
2,Robert,Smith
3,David,Smith
4,James,Johnson


In [9]:
fnames = list(np.unique(names['FirstName'].to_numpy()))
lnames = list(np.unique(names['Surname'].to_numpy()))

In [10]:
print(f'{len(fnames)} first names')
print(f'{len(lnames)} last names')

20 first names
20 last names


To generate 20 names, let's sample 4 last names and 5 first names

In [11]:
from random import sample

In [12]:
sample_fnames = sample(fnames, 5)
sample_lnames = sample(lnames, 4)

In [13]:
list(zip(sample_fnames, sample_lnames))

[('Susan', 'Rodriguez'),
 ('Linda', 'Miller'),
 ('Patricia', 'Wilson'),
 ('Daniel', 'Johnson')]

In [14]:
generated_names = []
for f in sample_fnames:
    for l in sample_lnames:
        generated_names.append(f + ' ' + l)

In [15]:
print(generated_names)

['Susan Rodriguez', 'Susan Miller', 'Susan Wilson', 'Susan Johnson', 'Linda Rodriguez', 'Linda Miller', 'Linda Wilson', 'Linda Johnson', 'Patricia Rodriguez', 'Patricia Miller', 'Patricia Wilson', 'Patricia Johnson', 'Daniel Rodriguez', 'Daniel Miller', 'Daniel Wilson', 'Daniel Johnson', 'David Rodriguez', 'David Miller', 'David Wilson', 'David Johnson']


In [16]:
from random import shuffle

In [17]:
shuffle(generated_names)

In [18]:
print(generated_names)

['Daniel Rodriguez', 'Daniel Miller', 'Patricia Johnson', 'Linda Rodriguez', 'Patricia Miller', 'Susan Johnson', 'Susan Miller', 'Susan Rodriguez', 'David Wilson', 'Daniel Wilson', 'Daniel Johnson', 'Susan Wilson', 'Linda Johnson', 'Patricia Rodriguez', 'Linda Wilson', 'David Miller', 'Linda Miller', 'David Rodriguez', 'Patricia Wilson', 'David Johnson']


## Partition into Leagues

In [19]:
league1 = generated_names[:10]
league2 = generated_names[10:]

In [20]:
league1

['Daniel Rodriguez',
 'Daniel Miller',
 'Patricia Johnson',
 'Linda Rodriguez',
 'Patricia Miller',
 'Susan Johnson',
 'Susan Miller',
 'Susan Rodriguez',
 'David Wilson',
 'Daniel Wilson']

In [21]:
league2

['Daniel Johnson',
 'Susan Wilson',
 'Linda Johnson',
 'Patricia Rodriguez',
 'Linda Wilson',
 'David Miller',
 'Linda Miller',
 'David Rodriguez',
 'Patricia Wilson',
 'David Johnson']

In [40]:
league = pd.DataFrame([[1, 'family'], [2, 'competitive']], columns=['leagueID', 'leagueType'])
league.set_index('leagueID')

,leagueType
leagueID,
1,family
2,competitive


In [41]:
league.to_csv('leagues.csv')

## Sample players to help form contracts

In [86]:
l1_qbs = sample(list(offense[offense['position'] == 'QB']['playerID'].unique()), len(league1))
l1_rbs = sample(list(offense[offense['position'] == 'RB']['playerID'].unique()), 2*len(league1))
l1_wrs = sample(list(offense[offense['position'] == 'WR']['playerID'].unique()), 2*len(league1))
l1_defense = sample(list(defense['playerID'].unique()), len(league1))

In [87]:
l2_qbs = sample(list(offense[offense['position'] == 'QB']['playerID'].unique()), len(league2))
l2_rbs = sample(list(offense[offense['position'] == 'RB']['playerID'].unique()), 2*len(league2))
l2_wrs = sample(list(offense[offense['position'] == 'WR']['playerID'].unique()), 2*len(league2))
l2_defense = sample(list(defense['playerID'].unique()), len(league2))

## Make contracts table

In [88]:
contract = pd.DataFrame(l1_qbs+l1_rbs+l1_wrs+l1_defense+l2_qbs+l2_rbs+l2_wrs+l2_defense, columns=['playerID'])

In [92]:
contract['owner'] = league1 + 2*league1 + 2*league1 + league1 + league2 + 2*league2 + 2*league2 + league2
contract['startWeek'] = 1
contract['endWeek'] = 16
contract['isActive'] = 1

In [93]:
contract

,playerID,owner,startWeek,endWeek,isActive
0,Jacoby Brissett,Daniel Rodriguez,1,16,1
1,Drew Brees,Daniel Miller,1,16,1
2,Jimmy Garoppolo,Patricia Johnson,1,16,1
3,Aaron Rodgers,Linda Rodriguez,1,16,1
4,Terrelle Pryor,Patricia Miller,1,16,1
...,...,...,...,...,...
115,Green Bay Packers,David Miller,1,16,1
116,Philadelphia Eagles,Linda Miller,1,16,1
117,Indianapolis Colts,David Rodriguez,1,16,1
118,Arizona Cardinals,Patricia Wilson,1,16,1


## Create Users table

In [108]:
user = pd.DataFrame([(name.split(' ')[0], name.split(' ')[1]) for name in generated_names], 
                    columns=['nameFirst', 'nameLast'])

In [111]:
user['userID'] = list(range(len(generated_names)))

In [112]:
user.head()

,nameFirst,nameLast,userID
0,Daniel,Rodriguez,0
1,Daniel,Miller,1
2,Patricia,Johnson,2
3,Linda,Rodriguez,3
4,Patricia,Miller,4


In [113]:
user.to_csv('users.csv')

## Create Teams table

In [117]:
league1

['Daniel Rodriguez',
 'Daniel Miller',
 'Patricia Johnson',
 'Linda Rodriguez',
 'Patricia Miller',
 'Susan Johnson',
 'Susan Miller',
 'Susan Rodriguez',
 'David Wilson',
 'Daniel Wilson']

In [120]:
leagues1 = {name: 1 for name in league1}
leagues2 = {name: 2 for name in league2}

In [122]:
leagues1.update(leagues2)

In [123]:
leagues1

{'Daniel Rodriguez': 1,
 'Daniel Miller': 1,
 'Patricia Johnson': 1,
 'Linda Rodriguez': 1,
 'Patricia Miller': 1,
 'Susan Johnson': 1,
 'Susan Miller': 1,
 'Susan Rodriguez': 1,
 'David Wilson': 1,
 'Daniel Wilson': 1,
 'Daniel Johnson': 2,
 'Susan Wilson': 2,
 'Linda Johnson': 2,
 'Patricia Rodriguez': 2,
 'Linda Wilson': 2,
 'David Miller': 2,
 'Linda Miller': 2,
 'David Rodriguez': 2,
 'Patricia Wilson': 2,
 'David Johnson': 2}

In [141]:
teams = pd.DataFrame([(f'Team{number}', name, leagues1[name]) for number, name in enumerate(generated_names)],
            columns=['teamID', 'userID', 'leagueID'])

In [142]:
teams.to_csv('teams.csv')

In [143]:
teams

,teamID,userID,leagueID
0,Team0,Daniel Rodriguez,1
1,Team1,Daniel Miller,1
2,Team2,Patricia Johnson,1
3,Team3,Linda Rodriguez,1
4,Team4,Patricia Miller,1
5,Team5,Susan Johnson,1
6,Team6,Susan Miller,1
7,Team7,Susan Rodriguez,1
8,Team8,David Wilson,1
9,Team9,Daniel Wilson,1


## Merge teams and contracts and drop names

In [129]:
contract

,playerID,owner,startWeek,endWeek,isActive
0,Jacoby Brissett,Daniel Rodriguez,1,16,1
1,Drew Brees,Daniel Miller,1,16,1
2,Jimmy Garoppolo,Patricia Johnson,1,16,1
3,Aaron Rodgers,Linda Rodriguez,1,16,1
4,Terrelle Pryor,Patricia Miller,1,16,1
...,...,...,...,...,...
115,Green Bay Packers,David Miller,1,16,1
116,Philadelphia Eagles,Linda Miller,1,16,1
117,Indianapolis Colts,David Rodriguez,1,16,1
118,Arizona Cardinals,Patricia Wilson,1,16,1


In [152]:
real_contracts = pd.merge(contract, teams.rename({'userID': 'owner'}, axis=1), on='owner').drop('owner', axis=1)

In [153]:
real_contracts.to_csv('contracts.csv')